# 关于函数
## 1 函数返回值个数大于3时
当函数返回值个数大于三个的时候，为了避免混乱，和方便后期调试，建议定义一个轻便的类或者namedtuple，并让函数返回这样的实例

## 2 try/except的异常处理不要只返回None
函数遇到意外情况，应该抛异常，而不是返回None

In [6]:
# 返回值尽量不要使用None
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

x, y = 5, 0
result = careful_divide(x, y)
# 这里可以区分返回值为0的情况，但是返回值一般是下面的判断方式
if result is None:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

x, y = 0, 5
result = careful_divide(x, y)
# 无法区分None和0
if not result:
    print('Invalid inputs')  # This runs! But shouldn't
else:
    assert False

Invalid inputs
Invalid inputs


In [4]:
# 返回值，第一表示是否成功，第二个表示计算结果
def careful_divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

x, y = 5, 0
success, result = careful_divide(x, y)
if not success:
    print('Invalid inputs')

x, y = 0, 5
# 但事实是，调用方往往会用下划线去忽略第一个值，此时又没法判断返回值时0的问题了
_, result = careful_divide(x, y)
if not result:
    print('Invalid inputs')

Invalid inputs
Invalid inputs


In [8]:
# 外部用try/except里层抛出的异常，并处理
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

x, y = 5, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Invalid inputs


In [32]:
# 加函数注解，并编写docstring来把有可能的异常写入文档里
def careful_divide(a: float, b: float) -> float: # 使用类型注解，规定返回值的类型
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

x, y = 2, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
    pass  # Expected

Invalid inputs


In [3]:
def func(a, b):
  return a, b
a, b = 1, 2
print(func(a,b))
a, b = func(a,b)
print(a,b)

def func(a, b, c):
  return (a,b,c)
a, b, c = 1, 2, 3
print(func(a,b,c))
a, b, c = func(a,b,c)
print(a,b,c)

(1, 2)
1 2
(1, 2, 3)
1 2 3


## 3 闭包作用域
- 闭包函数可以引用外部变量
- 闭包函数没法修改外部变量，需要使用nonlocal，模块内全局，区别于global的模块间全局
- 尽量不要使用全局变量，使用一个类包装一下比较好


In [36]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # Seems simple，这里新建一个内部局部变量然后赋值了，不影响外部的变量
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print('Found:', found) # False，出现了问题
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


In [34]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found  # Added
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority3(numbers, group)
print(found)
print(numbers)

True
[2, 3, 5, 7, 1, 4, 6, 8]


In [37]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter) # 调用了call方法
assert sorter.found is True
assert numbers == [2, 3, 5, 7, 1, 4, 6, 8]